# Momentum Screener

###### This notebook is used to develop a custom momentum-factor equities screener which will be used to find a selection of top momentum stocks based on the following criteria:



1.) Volatility Adjusted Momentum: We want to select equities where momentum exists even after adjusting for volatility or beta. That way we aren't just going long beta, we want to be compensated with excess returns for going long equities with relatively higher volatility.

2.) Momentum Quality: The path to momentum is important. We want to favour equities with smooth price returns rather than jumpy. Small but consisent price returns are more favorable as they are more discrete and attract less attention and have been studied to outperform for longer periods and improve risk adjusted momentum factor returns.
 - We Use the FIP (Frog in the Pan) Algorithm which looks at % negative days/ % positive days as a measure of momentum quality
 
3.) Convexity: We also will be screening for equities which display price convexity in their time series. We can think of convexity as the acceleration of returns, in other words price is increasing at an increasing rate
 - Quadratic Regression will give us a positive coefficient if there is convexity in the price data
 
Ranking Method: For simplicity we will be placing equal weight across the three criteria of vol adjusted returns, momentum quality, and convexity. What this means is that equities will be ranked on an aggregate score of each their 3 measured features... So you just add the index number for all 3 into the net score. 

The End result will be a CSV file that gets uploaded to our local computer consisting of a list of tickers and their score going from best to worst.
 
 
Bonus Tip:

Diversification: Diversification is key with momentum. Diverisify across market caps, and use multiple methods at ranking your equities.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.linear_model import LinearRegression
#import pandas_datareader as pdr

In [3]:
# Create a dataframe called 'sp500_tickers'
sp500_tickers = pd.read_csv('sp500_tickers.csv')  # Assuming you have a file of S&P500 tickers
sp500_tickers.head()

,ticker
0,AAPL
1,MSFT
2,GOOG
3,AMZN
4,TSLA


## Volatility Adjusted Returns

#### 1.) 1 Year Volatiltiy Adjusted Returns

In [7]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Vol Adjusted Return'])

In [9]:
# Collect rows in a list for concatenation
rows_to_add = []

for ticker in sp500_tickers['ticker']:  # Iterate over the 'ticker' column in sp500_tickers dataframe
    try:
        # Retrieve 1 year of historical data
        data = yf.download(ticker, period='2y')  
        
        # Check if data is insufficient
        if data.empty or len(data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        # Calculate daily returns
        data['Daily Return'] = data['Close'].pct_change()

        # Calculate percent return over a 1-year period (252 trading days)
        data['Percent Return'] = ((data['Close'] - data['Close'].shift(252)) / data['Close'].shift(252)) * 100  

        # Rolling window for std dev (daily returns)
        window = 252  

        # Calculate rolling standard deviation of daily returns
        data['Rolling_Std'] = data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        data['Rolling_Hist_Vol'] = data['Rolling_Std'] * np.sqrt(window)

        # Calculate volatility-adjusted returns
        data['Vol_Adjusted_Return'] = ((data['Percent Return'])/100) / (data['Rolling_Hist_Vol'])

        # Retrieve the last value of Vol Adjusted Return
        vol_adjusted_return = data['Vol_Adjusted_Return'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Vol Adjusted Return': vol_adjusted_return})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)

C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************

Error retrieving data for DFS: Insufficient data for ticker: DFS


[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t

Error retrieving data for CTLT: Insufficient data for ticker: CTLT


[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MRO']: YFPricesMissingError('possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")')
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\604339785.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Error retrieving data for HMENF: Insufficient data for ticker: HMENF
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [11]:
df = df.sort_values('Vol Adjusted Return', ascending=False)
# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

In [13]:
# read the csv file - momentum screen results
df1 = pd.read_csv("Momentum Screen Results.csv")

df1['Index1'] = df1.index # create column of the index position
df1.head()

,Ticker,Vol Adjusted Return,Index1
0,TPR,4.139843,0
1,MP,3.926470,1
2,HWM,2.924810,2
3,IBKR,2.850297,3
4,CAH,2.771468,4


### 2.) 6 Month Volatility Adjusted Returns

In [15]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Vol Adjusted Return'])

In [17]:
# Collect rows in a list for concatenation
rows_to_add = []

for ticker in sp500_tickers['ticker']:  # Iterate over the 'ticker' column in sp500_tickers dataframe
    try:
        # Retrieve 1 year of historical data
        data = yf.download(ticker, period='1y')  
        
        # Check if data is insufficient
        if data.empty or len(data) < 126:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        # Calculate daily returns
        data['Daily Return'] = data['Close'].pct_change()

        # Calculate percent return over a 1-year period (252 trading days)
        data['Percent Return'] = ((data['Close'] - data['Close'].shift(126)) / data['Close'].shift(126)) * 100  

        # Rolling window for std dev (daily returns)
        window = 126  

        # Calculate rolling standard deviation of daily returns
        data['Rolling_Std'] = data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        data['Rolling_Hist_Vol'] = data['Rolling_Std'] * np.sqrt(window)

        # Calculate volatility-adjusted returns
        data['Vol_Adjusted_Return'] = ((data['Percent Return'])/100) / (data['Rolling_Hist_Vol'])

        # Retrieve the last value of Vol Adjusted Return
        vol_adjusted_return = data['Vol_Adjusted_Return'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Vol Adjusted Return': vol_adjusted_return})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)

C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************

Error retrieving data for DFS: Insufficient data for ticker: DFS


[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t

Error retrieving data for CTLT: Insufficient data for ticker: CTLT


[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MRO']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')


...

[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t

Error retrieving data for NARI: Insufficient data for ticker: NARI


[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t

Error retrieving data for PDCO: Insufficient data for ticker: PDCO



C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\599058954.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='1y')
[*********************100%**********************

In [19]:
df = df.sort_values('Vol Adjusted Return', ascending=False)
# Create a csv file with the results
df.to_csv("6 Month Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

In [21]:
df2 = pd.read_csv("6 Month Momentum Screen Results.csv")
df2['Index2'] = df2.index # create column of the index position
df2.head()

,Ticker,Vol Adjusted Return,Index2
0,MP,2.486829,0
1,ORA,2.380375,1
2,NEU,2.249958,2
3,SXT,2.209822,3
4,APH,2.187099,4


### Merge Both Dataframes - 1yr and 6M vol volatility adjusted returns

In [23]:
# Perform an inner join to include only matching tickers
df = pd.merge(df1, df2, on='Ticker', how='inner')
df.head()

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2
0,TPR,4.139843,0,1.433532,39
1,MP,3.926470,1,2.486829,0
2,HWM,2.924810,2,1.392206,43
3,IBKR,2.850297,3,0.714457,152
4,CAH,2.771468,4,1.473877,36


### Calculate Average Rank

In [27]:
# calculate average rank

df['average_rank'] = df['Index1'] + df['Index2'] # simply add the two ranks
df.head()

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank
0,TPR,4.139843,0,1.433532,39,39
1,MP,3.926470,1,2.486829,0,1
2,HWM,2.924810,2,1.392206,43,45
3,IBKR,2.850297,3,0.714457,152,155
4,CAH,2.771468,4,1.473877,36,40


In [29]:
df = df.sort_values('average_rank', ascending=True)
# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index
df.head()


###############

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank
1,MP,3.926470,1,2.486829,0,1
9,NFG,2.570327,9,1.827960,7,16
17,CW,2.244402,17,1.972428,6,23
26,APH,1.995738,26,2.187099,4,30
27,ORCL,1.976552,27,1.762334,9,36


In [119]:
df = df.sort_values('Percent Return', ascending=False)

In [120]:
df.head()

,Ticker,Percent Return
8,WGS,2238.580136
6,PSIX,1910.499954
750,MNPR,1734.877246
1,RGTI,1277.227721
0,QUBT,1237.349470


In [121]:
# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

## Convexity of Returns

In [90]:
#only keep first 100 rows of the Momentum Screen Results 
df1 = df1.iloc[:100]

In [92]:
# Collect rows in a list for concatenation
rows_to_add_4 = []

# Initialize a results DataFrame
results_df = pd.DataFrame(columns=['Ticker', 'Convexity Ratio'])

for Ticker in df1['Ticker']:  # Iterate over the 'Ticker' column
    try:
        # Download 2 years of historical data
        data = yf.download(Ticker, period='2y')
        
        if data.empty or len(data) < 252:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {Ticker}")
        
        # Create a DataFrame for returns
        df = pd.DataFrame()
        df['Return'] = data['Adj Close'].pct_change()

        # Calculate the slope using a rolling window
        window = 50  # Adjust window size as needed
        df['Slope'] = df['Return'].rolling(window).apply(
            lambda x: np.polyfit(np.arange(len(x)), x, 1)[0], raw=True
        )

        # Compute the second derivative of the slope
        df['Second_Derivative'] = df['Slope'].diff()

        # Calculate the convexity ratio
        df['Convexity_Ratio'] = df['Second_Derivative'] / df['Slope']
        
        # Extract the most recent convexity ratio
        convexity_ratio = df['Convexity_Ratio'].iloc[-1]
        
        # Add the result to the list
        rows_to_add_4.append({'Ticker': Ticker, 'Convexity Ratio': convexity_ratio})
    
    except Exception as e:
        print(f"Error retrieving data for {Ticker}: {str(e)}")
        
# Concatenate all rows into the DataFrame at once
results_df = pd.concat([pd.DataFrame(rows_to_add_4)], ignore_index=True)

[*********************100%***********************]  1 of 1 completed


Error retrieving data for FOX: 'Adj Close'


[*********************100%***********************]  1 of 1 completed


Error retrieving data for FOXA: 'Adj Close'


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Error retrieving data for COKE: 'Adj Close'


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Error retrieving data for PGR: 'Adj Close'
Error retrieving data for TTWO: 'Adj Close'


KeyboardInterrupt: 

In [56]:
df2 = results_df.sort_values(by='Convexity Ratio', ascending=False)

In [79]:
df2.head()

,Ticker,Convexity Ratio
46,MRK,7.440757
48,INTC,3.696518
44,PEP,2.642901
24,TXN,1.623321
20,XOM,1.375793


In [83]:
#Create a new csv file with our FIP results
df2.to_csv('Convexity_Results.csv', index=False)

## Convexity Based on Quadratic Regression

In [172]:
df = df.iloc[:200]

In [174]:
rows_to_add_2 = []

# Initialize an empty DataFrame to store results
df_convexity = pd.DataFrame(columns=['Ticker', 'Convexity Score'])

for ticker in df['Ticker']:
    try:
        # Download historical data
        data = yf.download(ticker, period='2y')
        data = data.tail(252) # keep only last 252 rows... 252 trading days
        
        # Ensure there is sufficient data
        if data.empty or len(data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")
        
        # Calculate daily returns (percentage change)
        data['Return'] = data['Close'].pct_change() * 100

        # Drop the first row since the return for the first day is NaN
        data.dropna(inplace=True)
        
        # Create an ordinal time variable (index of the data)
        data.loc[:, 'Time'] = np.arange(len(data))
        
        # Prepare the independent variables (Time and Time^2 for quadratic regression)
        X = np.vstack([data['Time'], data['Time']**2]).T
        y = data['Return'].values
        
        # Perform a quadratic regression
        model = LinearRegression().fit(X, y)
        
        # The convexity is the coefficient of the squared time term
        convexity = model.coef_[1]  # This is the coefficient of Time^2 (quadratic term)
        
        # Add the result to the list
        rows_to_add_2.append({'Ticker': ticker, 'Convexity Score': convexity})
    
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")
        
# Concatenate all rows into the DataFrame at once
df_convexity = pd.concat([pd.DataFrame(rows_to_add_2)], ignore_index=True)

C:\Users\eabiz\AppData\Local\Temp\ipykernel_12240\2933314842.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_12240\2933314842.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_12240\2933314842.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_12240\2933314842.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='2y')
[*********************100%*******************

In [176]:
df_convexity = df_convexity.sort_values('Convexity Score', ascending=False)

In [178]:
df_convexity.reset_index(drop=True, inplace=True)

In [180]:
df_convexity.head()

,Ticker,Convexity Score
0,WGS,0.000051
1,OKE,0.000029
2,TATT,0.000023
3,TPR,0.000012
4,NI,0.000012


In [182]:
df_convexity.to_csv("Convexity_Score_Quadratic_Method.csv")


###

### Momentum Quality - FIP (Frog in the Pan Score)

ID or FIP Score
ID = sign x [% Negative - % Positive]

In [33]:
# Only keep first 400 rows of the Momentum Screen Results
df = df.iloc[:400]

In [35]:
# Create a blank dataframe with the appropriate columns
df_FIP = pd.DataFrame(columns=['Ticker', 'FIP_Score'])

In [37]:
rows_to_add_3 = []

for Ticker in df['Ticker']:  # Iterate over the 'ticker' column in sp500_ticker df
    try:
        data = yf.download(Ticker, period='2y')  # Retrieve data. Ensure > 252 days
        data = data.tail(252) # Use only last 252 rows... for 252 trading days
        
        if data.empty or len(data) < 252:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {ticker}")
        
        data['Percent Return'] = (data['Close'].pct_change())*100
        
        if data['Percent Return'].cumsum().iat[-1] > 0:
            sign = 1
        elif data['Percent Return'].cumsum().iat[-1] < 0:
            sign = -1
        else:
            sign = 0
            
        positive_days = (data['Percent Return'] > 0).sum()
        negative_days = (data['Percent Return'] < 0).sum()
        flat_days = (data['Percent Return'] == 0).sum()
        pct_positive = (round((positive_days/len(data)), 4))
        pct_negative = (round((negative_days/len(data)), 4))
        FIP_Score = round(sign*(pct_negative - pct_positive),4)
                
        # Add the result to the list
        rows_to_add_3.append({'Ticker': Ticker, 'FIP Score': FIP_Score})
        
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")
        
df_FIP = pd.concat([pd.DataFrame(rows_to_add_3)], ignore_index=True)

C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\4132541417.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(Ticker, period='2y')  # Retrieve data. Ensure > 252 days
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\4132541417.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(Ticker, period='2y')  # Retrieve data. Ensure > 252 days
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\4132541417.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(Ticker, period='2y')  # Retrieve data. Ensure > 252 days
[*********************100%***********************]  1 of 1 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_22488\4132541417.py:5: FutureWarning: YF.download() has changed argument auto_ad

In [39]:
df_FIP = df_FIP.sort_values('FIP Score', ascending=True) # ascending = True since the best scores are negative

In [41]:
df_FIP.reset_index(drop=True, inplace=True)

In [43]:
df_FIP.head()

,Ticker,FIP Score
0,DTE,-0.2301
1,NDAQ,-0.2222
2,CW,-0.2143
3,PM,-0.2103
4,EVRG,-0.2103


In [47]:
# Create a new csv file with our FIP results
df_FIP.to_csv('Momentum_FIP_Results.csv', index=False)


###

# Combine Dataframes and Rank Tickers

In [51]:
#Create column of index #
df['index_vol_rank'] = df.index #vol adjusted returns df

#Create column of index #
df_FIP['index_fip_rank'] = df_FIP.index # FIP df

# create column of index # 
#df_convexity['Index3'] = df_convexity.index

In [53]:
# Perform an inner join to include only matching tickers
merged = pd.merge(df_FIP, df, on='Ticker', how='inner')

In [55]:
merged.head()

,Ticker,FIP Score,index_fip_rank,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank,index_vol_rank
0,DTE,-0.2301,0,1.074054,127,1.428255,40,167,126
1,NDAQ,-0.2222,1,1.884202,33,0.957655,91,124,33
2,CW,-0.2143,2,2.244402,17,1.972428,6,23,17
3,PM,-0.2103,3,1.816799,39,1.378001,44,83,39
4,EVRG,-0.2103,4,1.715497,49,1.079955,70,119,49


### Next cell is only necessary if adding a third filter to the ranking:

In [39]:
# Merge the result with the third DataFrame
merged = pd.merge(merged, df_convexity, on='Ticker', how='inner')

NameError: name 'df_convexity' is not defined

In [58]:
merged.head()

,Ticker,FIP Score,index_fip_rank,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank,index_vol_rank
0,DTE,-0.2301,0,1.075585,127,1.424988,38,165,126
1,NDAQ,-0.2222,1,1.865993,34,0.949174,93,127,34
2,CW,-0.2143,2,2.203460,17,1.921033,6,23,17
3,PM,-0.2103,3,1.816159,39,1.374508,43,82,39
4,EVRG,-0.2103,4,1.721228,48,1.084006,69,117,48


In [57]:
# Calculate the score (sum of indices)

#merged['Score'] = merged['index_vol_rank'] + merged['index_fip_rank'] + merged['Index3']
merged['Score'] = merged['index_vol_rank'] + merged['index_fip_rank']
# Sort the result by the score
sorted_df = merged.sort_values('Score', ascending=True)
# Select relevant columns for the output
result = sorted_df[['Ticker', 'Score']]

In [59]:
result.head()

,Ticker,Score
5,CME,12
9,IBKR,12
10,CAH,14
15,HWM,17
2,CW,19


In [63]:
result.to_csv("Momentum_Ranked.csv",index=False)






#####